In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.preprocessing import PolynomialFeatures
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error, r2_score

In [9]:
df = pd.read_excel('/Users/mac/Desktop/учеба/проект/avito_data_main.xlsx')
df['views_by_day'] = df['page_views'] / df['days_active']


In [11]:
df.isna().sum()

item_id                                           0
category                                          0
days_active                                       0
has_video                                         0
photo_count                                       0
title                                             0
title_length                                      0
description_length                                0
description_word_count                            0
DescriptionRu                                     0
breed                                         34156
gender                                            0
age                                            8507
price                                             0
seller_n_month                                    0
seller_count_reviews                           1406
seller_rating                                  1513
page_views                                        0
contacts                                          0
favorites   

In [ ]:
def outlier(df, col):
    f_b = df[col].quantile(0.25)
    s_b = df[col].quantile(0.90)
    iqr = s_b - f_b
    lower = f_b - 3 * iqr
    higher = s_b + 3 * iqr
    df.drop(df[(df[col] < lower) | (df[col] > higher)].index, inplace=True)
    return df
    

In [30]:
outlier(df, 'views_by_day')

,item_id,category,days_active,has_video,photo_count,title,title_length,description_length,description_word_count,DescriptionRu,...,seller_count_reviews,seller_rating,page_views,contacts,favorites,conversion_page_views_contacts_%,conversion_page_views_favorites_%,conversion_page_views_favorites_contacts_%,sentiment,views_by_day
0,5940081728543144960,Кошки,5,False,4,Кот 2 года ко всему приучен,27,22,4,Кот не пакостит совсем,...,4.0,5.0,270,5,20,0.02,0.25,0.07,-0.894136,54.000000
1,6442878243383320576,Кошки,25,False,2,Кошка в дар бесплатно,21,50,11,без домная кошка мне 9 лет и мне очень жалко к...,...,0.0,0.0,19,0,0,0.00,NaN,0.00,-0.876198,0.760000
2,1007842829568802048,Кошки,2,False,4,Красивые здоровые кот и кошка 3 года,36,216,26,"Умерла хозяйка, бабушка, кошки никому не нужны...",...,23.0,5.0,33,0,1,0.00,0.00,0.03,-0.714857,16.500000
3,7914380850877753344,Кошки,25,False,2,Британская вислоухая,20,100,22,Девочка. Уже больше года. У ребенка оказалась ...,...,1.0,5.0,72,1,3,0.01,0.33,0.04,-0.710234,2.880000
4,6210833791942923264,Кошки,25,False,10,Шотландская вислоухая кошка,27,146,24,"Кошке 4 года, не стерилизованная, ни разу не к...",...,13.0,5.0,19,1,1,0.05,1.00,0.05,-0.673896,0.760000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
49968,5841270525617292288,Кошки,25,False,4,Кошки мейн-кун,14,21,3,Хорошая милая кошечка,...,0.0,0.0,19,0,0,0.00,NaN,0.00,0.995826,0.760000
49969,5406398888000874496,Кошки,25,False,1,Сибирская кошка,15,16,3,Красавица в дом!,...,2.0,5.0,56,1,1,0.02,1.00,0.02,0.997066,2.240000
49970,4649905965887619072,Кошки,11,False,5,Кот 3 года вислоухий,20,20,4,Кот красавец 3 года,...,1.0,1.0,5,0,0,0.00,NaN,0.00,0.997077,0.454545
49971,3002289748139043840,Кошки,25,False,1,Бенгальская кошка,17,5,1,Супер,...,0.0,0.0,36,0,0,0.00,NaN,0.00,0.999089,1.440000


In [31]:
df['views_by_day'].describe()

count    48804.000000
mean        10.716892
std         22.245769
min          0.000000
25%          0.590909
50%          2.000000
75%          8.666667
max        157.000000
Name: views_by_day, dtype: float64

In [32]:
X = df[['sentiment', 'description_word_count', 'photo_count']]
y = df['views_by_day']




In [33]:
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)

In [34]:
X_train, X_test, y_train, y_test = train_test_split(X_poly, y, test_size=0.2, random_state=42)

In [35]:
model = LinearRegression()
model.fit(X_train, y_train)

LinearRegression()

In [36]:
y_pred = model.predict(X_test)

In [37]:
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"MSE: {mse}")
print(f"R-squared: {r2}")

MSE: 498.4220212301531
R-squared: 0.009286014259723196


In [38]:
scores = cross_val_score(model, X_poly, y, cv=5, scoring='neg_mean_squared_error')
print(f"Cross-validation MSE scores: {-scores}")
print(f"Average cross-validation MSE: {-np.mean(scores)}")

Cross-validation MSE scores: [459.35990572 537.432408   535.27214855 476.3184232  475.99351816]
Average cross-validation MSE: 496.87528072629266


In [39]:
print("Coefficients:", model.coef_)
print("Intercept:", model.intercept_)

Coefficients: [ 2.54522792e+00 -5.30060741e-02  9.92822554e-03 -1.31006687e+01
 -2.19169094e-02  1.57183969e-01  1.07753204e-04 -1.87841219e-03
  3.66002479e-02]
Intercept: 12.442459581044197


In [1]:
df

NameError: name 'df' is not defined